# **distilbert-base-uncased基于TensorFlow做微调训练**

# 1.安装环境

In [ ]:
%%writefile requirements.txt
datasets
pyarrow
transformers
tensorflow
tf-keras
keras
torch

In [ ]:
pip install -r requirements.txt

# 2.加载数据

In [ ]:
from datasets import load_dataset

dataset = load_dataset("glue", "cola")
dataset = dataset["train"]  # Just take the training split for now

In [ ]:
import numpy as np
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("../models/distilbert-base-uncased")
tokenized_data = tokenizer(dataset["sentence"], return_tensors="np", padding=True)
# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_data = dict(tokenized_data)

labels = np.array(dataset["label"])  # Label is already an array of 0 and 1

In [ ]:
# def tokenize_dataset(data):
#     # Keys of the returned dictionary will be added to the dataset as columns
#     return tokenizer(data["text"])


# dataset = dataset.map(tokenize_dataset)

# 3.加载模型准备训练

In [ ]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("../models/distilbert-base-uncased", from_pt=True)
# Lower learning rates are often better for fine-tuning transformers
model.compile(optimizer=Adam(3e-5))  # No loss argument!
# model.compile(optimizer= 'adam' , loss= keras.losses.binary_crossentropy, metrics=['accuracy'])

model.fit(tokenized_data, labels)

In [ ]:
tf_dataset = model.prepare_tf_dataset(dataset["train"], batch_size=4, shuffle=True, tokenizer=tokenizer)

model.compile(optimizer=Adam(3e-5))  # No loss argument!

# 4.开始训练

In [ ]:
model.fit(tf_dataset)

# 5.测试训练结果

# 6.清理环境

In [ ]:
del model
del trained_model
del tokenizer

del dataset
del tokenized_datasets
del small_train_dataset
del small_eval_dataset

In [ ]:
pip uninstall -y -r requirements.txt

In [ ]:
rm -rf requirements.txt